In [2]:
#Strategy: Short Straddle

from jugaad_data.nse import NSELive
from time import sleep
from collections import defaultdict
from IPython.display import clear_output
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt


stock_list = ['PNB', 'BHEL','NTPC', "BEL","IOC","TATASTEEL"]
chosenmonths = ['Jan',]
sort_by = 'ROI' # 'Normal' or 'ROI'
months = ('Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec')

#Initiation
n = NSELive()

class Share():

    def __init__(self, symbol):
        self.symbol = symbol
        self.fno = n.stock_quote_fno(symbol)
        self.lotsize = self.fno['stocks'][0]['marketDeptOrderBook']['tradeInfo']['marketLot']
        self.currentprice = self.fno['stocks'][0]['underlyingValue']

        self.expiryDates = self.fno['expiryDates'].copy()
        self.isnifty = symbol in ('NIFTY', 'BANKNIFTY')
        self.investment = 0.50 * self.lotsize * self.currentprice



    def fno_refresh(self):
        self.fno = n.stock_quote_fno(self.symbol)
        self.currentprice = self.fno['stocks'][0]['underlyingValue']
        return self.fno

    def __str__(self):
        return self.symbol
shares = [ Share(stock) for stock in stock_list ]

def options_dict():
    return {'CALL' : 0, 'PUT' : 0, 'Volume' : [0, 0]}

#remember to refresh fno
while True:

    clear_output(wait=True)


    info = {
        share : defaultdict(options_dict) for share in shares
        }
    for share in shares:
        share.fno_refresh()
        for data in share.fno['stocks']:
            type = data['metadata']['optionType']
            if share.isnifty:
              expiry = data['metadata']['expiryDate'][:6]
            else:
              expiry = data['metadata']['expiryDate'][3:6]
            expiryMonth = data['metadata']['expiryDate'][3:6]
            strike = data['metadata']['strikePrice']
            if (
                  'Options' in data['metadata']['instrumentType']
            and ( type == 'Call' or type == 'Put' )
            and ( 0.98 * share.currentprice ) <=  strike
            and ( 1.05 * share.currentprice ) >=  strike
            and ( expiryMonth in chosenmonths )
            ):
                if type == 'Call':
                    info[share][f'{expiry}{strike}']['CALL'] = data['metadata']['lastPrice']
                    info[share][f'{expiry}{strike}']['Volume'][0] = data['metadata']['numberOfContractsTraded']
                if type == 'Put':
                    info[share][f'{expiry}{strike}']['PUT'] = data['metadata']['lastPrice']
                    info[share][f'{expiry}{strike}']['Volume'][1] = data['metadata']['numberOfContractsTraded']


    #vinfo = symbol, cureentprice, expiry, strike, CALL premium, PUT premium, added, safety, investment, MAX Profit, MAX ROI, CALL Volume, PUT Volume

    vinfo=[]
    for share, data in info.items():
        if share.isnifty:
            for strike, ps in data.items():
                vinfo.append( [share.symbol, share.currentprice, strike[0:6], float(strike[6:]), ps["CALL"], ps["PUT"], round(ps["CALL"]+ps["PUT"], 2), round(float(strike[6:]) - ps["CALL"] - ps["PUT"], 1), round(float(strike[6:]) + ps["CALL"] + ps["PUT"],1), share.investment, (ps["CALL"]+ps["PUT"]) * share.lotsize, round( (ps["CALL"]+ps["PUT"])*share.lotsize/share.investment * 100  , 2), ps['Volume'][0], ps['Volume'][1] ] )
        else:
            for strike, ps in data.items():
                vinfo.append([share.symbol, share.currentprice, strike[0:3], float(strike[3:]), ps["CALL"], ps["PUT"], round(ps["CALL"]+ps["PUT"], 2), round(float(strike[3:]) - ps["CALL"] - ps["PUT"], 1), round(float(strike[3:]) + ps["CALL"] + ps["PUT"],1), share.investment, (ps["CALL"]+ps["PUT"]) * share.lotsize, round( (ps["CALL"]+ps["PUT"])*share.lotsize/share.investment * 100  , 2), ps['Volume'][0], ps['Volume'][1] ])

    if sort_by=='Normal':
        vinfo.sort(key = lambda x: (x[0], months.index(x[2][-3:]), x[3]))
    elif sort_by=='ROI':
        vinfo.sort(key = lambda x : x[-3], reverse=True)

    df = pd.DataFrame(vinfo, columns=['Share', 'CM Price', 'Expiry', 'SP', 'CALL', 'PUT', 'C+P', 'Short Safety', 'Long Safety', 'Investment', 'Max Profit', 'Max ROI', 'CALL Vol', 'PUT Vol'])
    print(df[['Share', 'Expiry', 'CM Price', 'Short Safety', 'Long Safety', 'Investment', 'SP', 'CALL', 'PUT', 'C+P', 'Max Profit', 'Max ROI', 'CALL Vol', 'PUT Vol']].to_string())
    print(datetime.now())

    if len(chosenmonths) == 1 and len(stock_list) == 1:
        plt.figure(figsize=(8, 6))
        plt.plot(
            df['SP'],
            df['SP'],
            marker='o',
            markersize='6'
        )
        plt.plot(
            df['SP'],
            df['CM Price']
        )
        plt.fill_between(df['SP'], df['Long Safety'], df['Short Safety'], alpha=0.3,color='b')

        for i in range(len(df)):
            plt.text(df['SP'][i], df['SP'][i] + 0.5, str(df['C+P'][i]), ha='center')

        plt.xlabel('Strike Price')
        plt.ylabel('Safety Range')
        plt.title('PNB')
        plt.legend()
        plt.grid(True)
        plt.show()


    sleep(15)

IndexError: list index out of range

In [ ]:
#Strategy: Buy an option and short the Call for that (I don't remember the name)
#Remember: The upside is completely safe, but it doesn't provide much safety on the down side. The strategy works as shown above

from jugaad_data.nse import NSELive
import pandas as pd
from time import sleep
from IPython.display import clear_output
from datetime import datetime

#Initiation
n = NSELive()
stock_list = ['PNB', 'SBIN','BEL']
chosenmonths = ['Jan',]


while True:

    # Clear the current cell's output
    clear_output(wait=True)

    #Fetching data from the net
    fno = [ n.stock_quote_fno(x) for x in stock_list ]
    cm = [ n.stock_quote(x) for x in stock_list ]

    #Printing everything about call options
    """
    for stockfno in fno:
        for i in stockfno['stocks']:
            if i['metadata']['instrumentType'] == 'Stock Options' and i['metadata']['optionType'] == 'Call':
                for x,y in i.items():
                    print(f'{x} : {y}')
                print('\n')
    """

    #A few variable from CM
    expiryDates = list(set(fno[0]['expiryDates'].copy()))
    months = ('Jan', 'Feb', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec')

    lots = { share['info']['symbol'] : share['stocks'][0]['marketDeptOrderBook']['tradeInfo']['marketLot'] for share in fno }
    lastprice = { share['info']['symbol'] : share['priceInfo']['lastPrice'] for share in cm }

    #Calculations and arranging data
    #( symbol, current, lot, investment, interest, expiry, strike, premium, maxp, maxroi, minp, minroi, zero )
    v_info = []

    for share in fno:
        current_price = lastprice[share['info']['symbol']]
        lotsize = lots[share['info']['symbol']]
        for data in share['stocks']:
            if (
            data['metadata']['instrumentType'] == 'Stock Options'
            and data['metadata']['optionType'] == 'Call'
            and (0.999 * lastprice[share['info']['symbol']] ) <=  data['metadata']['strikePrice']
            and (1.06 * lastprice[share['info']['symbol']] ) >=  data['metadata']['strikePrice']
            and (data['metadata']['expiryDate'][3:6] in chosenmonths)
        ):
                strike = data['metadata']['strikePrice']
                premium = data['metadata']['lastPrice']
                inves = int( current_price * lotsize / 1.5 )
                interest = round( 0.01 * current_price * lotsize / 1.5, 2 )

                v_info.append((
                        share['info']['symbol'],
                        current_price,
                        lotsize,
                        inves,
                        interest,
                        data['metadata']['expiryDate'][3:6],
                        strike,
                        premium,
                        int( ( ( strike - current_price + premium ) * lotsize ) - interest),
                        round( 100 * ( ( ( strike - current_price + premium ) * lotsize ) - interest ) / inves , 2 ),
                        int( ( premium * lotsize ) - interest ),
                        round( 100 * (( premium * lotsize ) - interest) / inves , 2 ),
                        round( (1.003 * current_price ) - premium , 4),
                        round( ((premium - (0.003 * current_price))/current_price * 100) , 2)
                    ))


                v_info.sort(key = lambda x : ( x[0] , months.index(x[5]), x[6] ))


    #Data Display
    df = pd.DataFrame(v_info, columns=['Share', 'CM Price', 'Lot Size', 'Investment', 'Interest', 'Expiry', 'SP', 'Premium', 'Max Profit', 'Max ROI %', 'OP Profit', 'OP ROI %', 'Zero Point', 'Zero Point %'])

    print(df[[ 'Share', 'Expiry', 'CM Price', 'Investment', 'SP', 'Premium', 'Max ROI %', 'OP ROI %', 'Max Profit', 'OP Profit', 'Zero Point', 'Zero Point %']].to_string())

    print(datetime.now())

    sleep(15)

IndexError: list index out of range